# Pandas: DataFrame apjungimas (Combining DataFrames)

Faile pateikiami aiškūs ir nuoseklūs pavyzdžiai, kaip su `pandas`:

- apjungti lenteles vertikaliai (appending)
- apjungti lenteles horizontaliai (joining / merging)
- suprasti pagrindinius join tipus (inner, left ir pan.)
- naudoti `join()` metodą

Pavyzdžiai orientuoti į duomenų analitikos scenarijų (užsakymai, klientai, produktai).
Komentarai pateikiami techniniu, aiškinamuoju stiliumi.


In [1]:
import pandas as pd

pd.__version__


'2.3.3'

## 1. Pavyzdiniai duomenys

Naudojamos trys paprastos lentelės:
- klientai (customers)
- užsakymai (orders)
- produktai (products)

Tokio tipo duomenys dažnai sutinkami analizėje ir BI paruošime.


In [2]:
customers = pd.DataFrame({
    "customer_id": [1, 2, 3, 4],
    "customer_name": ["Asta", "Mantas", "Ieva", "Tomas"],
    "segment": ["Retail", "Retail", "Business", "Retail"]
})

orders = pd.DataFrame({
    "order_id": [101, 102, 103, 104, 105],
    "customer_id": [1, 2, 2, 3, 5],  # customer_id=5 neturi atitikmens customers lentelėje
    "product_id": [10, 10, 11, 12, 12],
    "quantity": [1, 2, 1, 3, 1]
})

products = pd.DataFrame({
    "product_id": [10, 11, 12],
    "product_name": ["Bike", "Helmet", "Bottle"],
    "unit_price": [800, 50, 12]
})

customers, orders, products


(   customer_id customer_name   segment
 0            1          Asta    Retail
 1            2        Mantas    Retail
 2            3          Ieva  Business
 3            4         Tomas    Retail,
    order_id  customer_id  product_id  quantity
 0       101            1          10         1
 1       102            2          10         2
 2       103            2          11         1
 3       104            3          12         3
 4       105            5          12         1,
    product_id product_name  unit_price
 0          10         Bike         800
 1          11       Helmet          50
 2          12       Bottle          12)

## 2. Appending DataFrames (vertikalus sujungimas)

Vertikalus sujungimas reiškia, kad lentelės su ta pačia struktūra „sukraunamos“ viena po kitos.
Tam naudojamas `pd.concat()`.

Dažna klaida:
- naudoti `DataFrame.append()` (nebenaudojama ir dažnai lėta)
- pamiršti `ignore_index=True`, todėl indeksai kartojasi
- bandyti sujungti lenteles su skirtingais stulpeliais ir tikėtis „tvarkingo“ rezultato


In [5]:
jan_sales = pd.DataFrame({
    "month": ["2025-01", "2025-01"],
    "store": [1, 2],
    "revenue": [1200, 900]
})

feb_sales = pd.DataFrame({
    "month": ["2025-02", "2025-02"],
    "store": [1, 2],
    "revenue": [1500, 1100],
    "taxes": [1500, 1100]
})

combined_sales = pd.concat([jan_sales, feb_sales], ignore_index=True)
combined_sales


,month,store,revenue,taxes
0,2025-01,1,1200,NaN
1,2025-01,2,900,NaN
2,2025-02,1,1500,1500.0
3,2025-02,2,1100,1100.0


### Appending su skirtingais stulpeliais

Jei stulpeliai skiriasi, `concat` sukurs trūkstamas reikšmes (`NaN`).
Tai gali būti priimtina, bet svarbu suprasti rezultatą.


In [6]:
extra_cols = pd.DataFrame({
    "month": ["2025-03"],
    "store": [1],
    "revenue": [1400],
    "promo": [True]
})

combined_mixed = pd.concat([combined_sales, extra_cols], ignore_index=True)
combined_mixed


,month,store,revenue,taxes,promo
0,2025-01,1,1200,NaN,NaN
1,2025-01,2,900,NaN,NaN
2,2025-02,1,1500,1500.0,NaN
3,2025-02,2,1100,1100.0,NaN
4,2025-03,1,1400,NaN,True


## 3. Joining DataFrames (horizontalus sujungimas)

Horizontalus sujungimas reiškia, kad lentelės apjungiamos pagal raktą (key).
Dažniausiai naudojami metodai:
- `merge()` – klasikinis SQL join analogas
- `join()` – sujungimas pagal indeksą (arba pagal raktą, jei jis paverstas indeksu)

Dažna klaida:
- sujungti pagal neteisingą raktą
- turėti dublikatus rakte ir gauti padaugintas eilutes
- pasirinkti `left` / `inner` tipą neįvertinus, ką reiškia prarastos arba atsiradusios eilutės


## 4. Join tipai (SQL logika)

- **Inner join**: paliekamos tik tos eilutės, kurios turi atitikmenį abiejose lentelėse.
- **Left join**: paliekamos visos kairės lentelės eilutės, o trūkstami atitikmenys dešinėje tampa `NaN`.

Analitiko praktikoje `left join` dažnai naudojamas kaip „pagrindinės lentelės“ praturtinimas papildomais atributais.


In [7]:
# INNER JOIN: užsakymai + klientai (paliekami tik tie užsakymai, kurių klientas egzistuoja customers lentelėje)
orders_inner = orders.merge(customers, on="customer_id", how="inner")
orders_inner


,order_id,customer_id,product_id,quantity,customer_name,segment
0,101,1,10,1,Asta,Retail
1,102,2,10,2,Mantas,Retail
2,103,2,11,1,Mantas,Retail
3,104,3,12,3,Ieva,Business


In [8]:
# LEFT JOIN: paliekami visi užsakymai, net jei klientas nerandamas
orders_left = orders.merge(customers, on="customer_id", how="left")
orders_left


,order_id,customer_id,product_id,quantity,customer_name,segment
0,101,1,10,1,Asta,Retail
1,102,2,10,2,Mantas,Retail
2,103,2,11,1,Mantas,Retail
3,104,3,12,3,Ieva,Business
4,105,5,12,1,NaN,NaN


Dažna klaida:
- nepastebėti `NaN` po left join ir vėliau bandyti naudoti tuos laukus skaičiavimuose
- manyti, kad inner join „teisingesnis“, nors jis gali išmesti realius įrašus (pvz., klientai neegzistuoja master lentelėje)


## 5. `join()` metodas

`join()` patogiausias, kai:
- raktas yra indeksas
- reikia pridėti stulpelius iš kitos lentelės pagal indeksą

Dažna klaida:
- bandyti naudoti `join()` neparuošus indekso ir gauti netikėtus rezultatus


In [9]:
# Paruošiamas indeksas
customers_idx = customers.set_index("customer_id")

# join pagal customer_id (iš orders pusės)
orders_join = orders.join(customers_idx, on="customer_id", how="left")
orders_join


,order_id,customer_id,product_id,quantity,customer_name,segment
0,101,1,10,1,Asta,Retail
1,102,2,10,2,Mantas,Retail
2,103,2,11,1,Mantas,Retail
3,104,3,12,3,Ieva,Business
4,105,5,12,1,NaN,NaN


## 6. Apjungimas su daugiau lentelių (praktiškas pavyzdys)

Dažna analitikos situacija:
- užsakymų lentelė su kiekiu
- produktų lentelė su kainomis
- reikia paskaičiuoti pajamas

Svarbi praktika:
- po sujungimo pasitikrinti eilučių skaičių (`shape`)
- patikrinti, ar raktai unikalūs ten, kur to tikimasi


In [10]:
orders_enriched = (
    orders
    .merge(customers, on="customer_id", how="left")
    .merge(products, on="product_id", how="left")
)


orders_enriched



,order_id,customer_id,product_id,quantity,customer_name,segment,product_name,unit_price
0,101,1,10,1,Asta,Retail,Bike,800
1,102,2,10,2,Mantas,Retail,Bike,800
2,103,2,11,1,Mantas,Retail,Helmet,50
3,104,3,12,3,Ieva,Business,Bottle,12
4,105,5,12,1,NaN,NaN,Bottle,12


In [11]:
orders_enriched["revenue"] = (orders_enriched["quantity"] * orders_enriched["unit_price"]).round(2)

orders_enriched

,order_id,customer_id,product_id,quantity,customer_name,segment,product_name,unit_price,revenue
0,101,1,10,1,Asta,Retail,Bike,800,800
1,102,2,10,2,Mantas,Retail,Bike,800,1600
2,103,2,11,1,Mantas,Retail,Helmet,50,50
3,104,3,12,3,Ieva,Business,Bottle,12,36
4,105,5,12,1,NaN,NaN,Bottle,12,12


In [12]:
orders_enriched.shape

(5, 9)

## 7. Dažnos klaidos ir gerosios praktikos (santrauka)

Dažnos klaidos:
- naudoti `append()` vietoje `pd.concat()`
- pamiršti `ignore_index=True` vertikaliam sujungimui
- sujungti pagal neteisingą raktą
- turėti dublikatus rakte ir gauti padaugintas eilutes
- ignoruoti `NaN` po left join

Gerosios praktikos:
- `pd.concat()` naudoti appending atvejais
- `merge()` naudoti SQL tipo join logikai
- `join()` naudoti, kai raktas yra indeksas
- po apjungimo tikrinti `shape` ir `isna().sum()`
- aiškiai pasirinkti `inner` arba `left` pagal analizės tikslą
